# 🎭 KLUE-BERT 감정 분류 학습\n\n**목표**: 100 Epochs로 78%+ 정확도 달성\n- 데이터: 41,387개\n- GPU: Colab T4

In [ ]:
# GPU 확인\n!nvidia-smi

In [ ]:
# 저장소 클론\n!git clone https://github.com/gijung2/chatbot.git\n%cd chatbot

In [ ]:
# 패키지 설치\n!pip install -q transformers torch pandas scikit-learn matplotlib seaborn tqdm

## 학습 시작 (10 Epochs 테스트)

In [ ]:
!python training/main_kfold.py --data_path data/processed/emotion_corpus_full.csv --model_name klue/bert-base --k_folds 5 --epochs 10 --batch_size 32 --output_dir checkpoints_kfold